<h1 id="exercises">Exercises - Data Acquisition</h1>


This exercises uses the <code>case.csv</code>, <code>dept.csv</code>, and <code>source.csv</code> files from the
san antonio 311 call dataset.

In [1]:
import pandas as pd
import numpy as np

from os import path
from pydataset import data

import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
def get_spark_shape(df):
    return (df.count(), len(df.columns))

## i.1
Read the case, department, and source data into their own spark dataframes.

#### Create case

In [4]:
case=spark.read.csv('spark_csv/case.csv', header=True, inferSchema=True)
print(get_spark_shape(case))
case.printSchema()

(841704, 14)
root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



#### Create dept

In [5]:
dept=spark.read.csv('spark_csv/dept.csv', header=True, inferSchema=True)
print(get_spark_shape(dept))
dept.printSchema()

(39, 4)
root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



#### Create source

In [6]:
source=spark.read.csv('spark_csv/source.csv', header=True, inferSchema=True)
print(get_spark_shape(source))
source.printSchema()

(140, 2)
root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



## i.2
Let's see how writing to the local disk works in spark:

### i.2.a
Write the code necessary to store the source data in both csv and json format, store these as <code>sources_csv</code> and <code>sources_json</code>

In [7]:
csv_file = 'data/sources_csv'

source.write.csv(csv_file, mode='overwrite')
if path.exists(csv_file):
    print('file saved')

file saved


In [8]:
json_file = 'data/sources_json'

source.write.json(json_file, mode='overwrite')
if path.exists(json_file):
    print('file saved')

file saved


### i.2.b
Inspect your folder structure. What do you notice?

**Spark makes directories, not just files.**

## i.3
Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

#### inspect case

In [9]:
case.show(3, vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 1/1/18 0:42                           
 case_closed_date     | 1/1/18 12:29                          
 SLA_due_date         | 9/26/20 0:42                          
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | YES                                   
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
-RECORD 1----------------------------------------------

#### Modify case

- Change 'SLA' to 'case
- Set dates to timestamps
- Set YES/NO fields to boolean
- Change council district field to string

In [10]:
fmt = "M/d/yy H:mm"
case = (case
        .withColumnRenamed('SLA_due_date', 'case_due_date')
        .withColumnRenamed('SLA_days', 'case_days')
        .withColumn("case_opened_date", F.to_timestamp("case_opened_date", fmt))
        .withColumn("case_closed_date", F.to_timestamp("case_closed_date", fmt))
        .withColumn("case_due_date", F.to_timestamp("case_due_date", fmt))
        .withColumn("case_closed", F.expr('case_closed == "YES"'))
        .withColumn("case_late", F.expr('case_late == "YES"'))
        .withColumn("council_district", F.col("council_district").cast("string"))
)
case.printSchema()
case.show(3, vertical=True, truncate=False)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- case_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 2018-01-01 00:42:00                   
 case_closed_date     | 2018-01-01 12:29:00                   
 case_due_date        | 2020-09-26 00:42:00                   
 case_late            | false            

#### Register case

In [11]:
case.createOrReplaceTempView('case')

#### Inspect dept

In [12]:
dept.show(3, truncate=False)

+---------------+----------------------+----------------------+-------------------+
|dept_division  |dept_name             |standardized_dept_name|dept_subject_to_SLA|
+---------------+----------------------+----------------------+-------------------+
|311 Call Center|Customer Service      |Customer Service      |YES                |
|Brush          |Solid Waste Management|Solid Waste           |YES                |
|Clean and Green|Parks and Recreation  |Parks & Recreation    |YES                |
+---------------+----------------------+----------------------+-------------------+
only showing top 3 rows



#### Modify dept

- Change dept_subject_to_SLA to boolean

In [13]:
dept = (dept
        .withColumn("dept_subject_to_SLA", F.expr('dept_subject_to_SLA == "YES"'))
        
)
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



#### Register dept

In [14]:
dept.createOrReplaceTempView('dept')

#### Inspect source

In [15]:
source.show(3, truncate=False)

+---------+----------------+
|source_id|source_username |
+---------+----------------+
|100137   |Merlene Blodgett|
|103582   |Carmen Cura     |
|106463   |Richard Sanchez |
+---------+----------------+
only showing top 3 rows



#### Modify source

* No modifications necessary *

#### Register source

In [16]:
source.createOrReplaceTempView('source')

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [17]:
spark.sql('''
    SELECT MAX(case_opened_date) MaxOpen, MAX(case_closed_date) MaxClose FROM case
''').show()

+-------------------+-------------------+
|            MaxOpen|           MaxClose|
+-------------------+-------------------+
|2018-08-08 10:38:00|2018-08-08 10:38:00|
+-------------------+-------------------+



In [50]:
spark.sql('''
    SELECT 
        date_add(case_opened_date, 0) case_opened, 
        date_add(case_due_date, 0) case_due, 
        date_add(case_closed_date, 0) case_closed, 
        case_days, 
        num_days_late, 
        case_days + num_days_late case_age,
--        date_add(case_due_date, num_days_late) calcdue, 
        date_add(case_opened_date, round(case_days + num_days_late)) calcage
    FROM 
        case
    WHERE 
        (case_closed = True 
            AND 
            date_add(case_opened_date, round(case_days + num_days_late)) <> date_add(case_closed_date, 0))
        OR
        (case_closed = False
            AND
            date_add(case_opened_date, int(case_days) + int(num_days_late)) <> '2018-08-08')
''').show()

+-----------+----------+-----------+-----------+-------------------+------------------+----------+
|case_opened|  case_due|case_closed|  case_days|      num_days_late|          case_age|   calcage|
+-----------+----------+-----------+-----------+-------------------+------------------+----------+
| 2018-01-01|2018-01-11| 2018-01-03|       10.0|       -8.580949074|1.4190509260000006|2018-01-02|
| 2018-01-01|2018-01-17| 2018-01-02|15.41878472|-15.038703699999997|0.3800810200000022|2018-01-01|
| 2018-01-01|2018-01-02| 2018-01-02|      0.125|         0.34244213|        0.46744213|2018-01-01|
| 2018-01-02|2018-01-17| 2018-01-02|15.30795139|-14.647546300000002|0.6604050899999976|2018-01-03|
| 2018-01-02|2018-01-17| 2018-01-02|15.30527778|       -14.77136574|0.5339120400000006|2018-01-03|
| 2018-01-02|2018-01-17| 2018-01-02|15.30206019|        -14.6334375|0.6686226900000012|2018-01-03|
| 2018-01-02|2018-01-17| 2018-01-02|15.30078704|       -14.63961806|0.6611689799999994|2018-01-03|
| 2018-01-

## ii.1
How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

## ii.2
How many Stray Animal cases are there?

## ii.3
How many service requests that are assigned to the Field Operations department (<code>dept_division</code>) are not classified as "Officer Standby" request type (<code>service_request_type</code>)?

## ii.4
Convert the <code>council_district</code> column to a string column.

## ii.5
Extract the year from the <code>case_closed_date</code> column.


## ii.6
Convert <code>num_days_late</code> from days to hours in new columns <code>num_hours_late</code>.

## ii.7
Join the case data with the source and department data.

## ii.8
Are there any cases that do not have a request source?

## ii.9
What are the top 10 service request types in terms of number of requests?

## ii.10
What are the top 10 service request types in terms of average days late?

## ii.11
Does number of days late depend on department?

## ii.12
How do number of days late depend on department and request type?